# Used Functions

In [1]:
def fixed_point_to_binary(fixed_point_data, num_bits, fractional_bits):
    # Calculate the maximum integer value that can be represented with the given number of bits
    max_int_value = 2**(num_bits - 1) - 1
    
    # Scale the fixed-point data to fit within the representable integer range
    scaled_data = int(fixed_point_data * (2**fractional_bits))
    
    # Clip the scaled data to stay within the representable integer range
    clipped_data = max(-max_int_value, min(scaled_data, max_int_value))
    
    # Convert the clipped data to binary representation
    binary_representation = bin(clipped_data & int("1"*num_bits, 2))[2:].zfill(num_bits)
    
    return binary_representation

# Base Video Pipeline

## Pipeline

Load overlay

In [3]:
from pynq import Overlay
from pynq.lib.video import *

base = Overlay('/home/xilinx/jupyter_notebooks/contrast/base.bit')
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

from pynq import MMIO
contrast_addr = 0x4000_0000
pre_sharpen_addr = 0x4001_0000
post_sharpen_addr = 0x4002_0000
contrast = MMIO(contrast_addr, 0x10000)
pre_sharpen = MMIO(pre_sharpen_addr, 0x10000)
post_sharpen = MMIO(post_sharpen_addr, 0x10000)



In [32]:
def set_constrast(scaler):
    contrast.write(0x10, int(fixed_point_to_binary(scaler, 32, 16), 2))
                    
def toggle_contrast():
    status = contrast.read(0x18)
    contrast.write(0x18, not status)
                   
def toggle_pre_sharpen():
    status = pre_sharpen.read(0x10)
    pre_sharpen.write(0x10, not status)
                   
def toggle_post_sharpen():
    status = post_sharpen.read(0x10)
    post_sharpen.write(0x10, not status)

Configure HDMI

In [4]:
hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

Write configuration data (for the `invstripe` function)

In [56]:
set_constrast(0.9)

In [23]:
contrast.write(0x18, 0)

In [33]:
pre_sharpen.write(0x10, 1)

In [54]:
post_sharpen.write(0x10,0)

Don't forget to close the HDMI connection!

In [ ]:
hdmi_out.close()
hdmi_in.close()